# **S3_App Subscription, Dashboard, and Low App Performers**

In [76]:
import os
import pandas as pd
from openpyxl import Workbook, load_workbook
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.styles import PatternFill
from openpyxl.styles import Font
from openpyxl.styles import Border, Side
import shutil
from datetime import datetime, timedelta
import xlwings as xw

In [77]:
deployment=input('Enter the name of ACO ')
date_input = input ('Enter the date frame as mmdd-mmdd: ')
path=fr"C:\Users\ZemingLiu\OneDrive - PBACO\Documents\Weekly App Reports\Derrell Macros\Patient Alerts Macro\{date_input} {deployment}"

#function to get last week
def get_previous_week(current_week):
    start_date_str, end_date_str = current_week.split('-')
    start_date = datetime.strptime(start_date_str, "%m%d")
    end_date = datetime.strptime(end_date_str, "%m%d")
    prev_start_date = start_date - timedelta(days=7)
    prev_end_date = end_date - timedelta(days=7)
    lastweek_date_input = f"{prev_start_date.strftime('%m%d')}-{prev_end_date.strftime('%m%d')}"
    return lastweek_date_input
lastweek_date_input = get_previous_week(date_input)

#import subscription and alert_working
subscription=pd.read_excel(fr'{path}\App Subscription.xlsx',skiprows=3)
if deployment == 'PBACO':
    df=pd.read_excel(fr'{path}\patient_alerts_working_{date_input}.xlsx',skiprows=5,sheet_name='All')
else:
    df=pd.read_excel(fr'{path}\patient_alerts_working_{deployment}_{date_input}.xlsx',skiprows=5,sheet_name='All')

Enter the name of ACO  Empire
Enter the date frame as mmdd-mmdd:  1223-1229


# **Part 1 App Subscription**

In [78]:
#drop empty columns
drop_col=[col for col in subscription.columns if 'Unnamed' in col] 
subscription.drop(drop_col,axis=1,inplace=True)

#drop rows not in the alert_working
drop_rows = subscription[~subscription['NPI'].isin(df['NPI'])].index
subscription.drop(drop_rows,axis=0,inplace=True)

#concat npi name
subscription.insert(3,'NPI Name',(subscription['NPI FirstName']+', '+subscription['NPI LastName']))
subscription.drop(subscription.columns[4:6], axis=1, inplace=True)

In [79]:
#add a column as highlight indicator, highlight cell if NPI has no patient read
subscription['highlight_indicator']=''
highlight_NPI = df[df['Total Pts Read']==0]['NPI'].tolist()
subscription['highlight_indicator'] = subscription['NPI'].isin(highlight_NPI).astype(int)    
#sort
subscription = subscription.sort_values(by=['highlight_indicator', 'Entity', 'NPI Name'], ascending=[False, True, True])

In [80]:
#save as excel then use openxl import again
if deployment == 'PBACO':
    subscription.to_excel(fr'{path}\patient_alerts_{date_input}\App Subscription_{date_input}.xlsx',index=False)
    wb=load_workbook(fr'{path}\patient_alerts_{date_input}\App Subscription_{date_input}.xlsx')
else:
    subscription.to_excel(fr'{path}\patient_alerts_{deployment}_{date_input}\App Subscription_{deployment}_{date_input}.xlsx',index=False)
    wb=load_workbook(fr'{path}\patient_alerts_{deployment}_{date_input}\App Subscription_{deployment}_{date_input}.xlsx')
ws=wb.active

In [81]:
#highlight NPI column
for row in range(2, ws.max_row + 1): 
    cell = ws[f"C{row}"]
    if ws[f"L{row}"].value ==1:
        cell.fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")

#drop indicator column
ws.delete_cols(12)

#convert to table
maxrow=ws.max_row
minrow=ws.min_row
maxcol=ws.max_column
mincol=ws.min_column
table_range = f"{ws.cell(row=minrow, column=mincol).coordinate}:{ws.cell(row=maxrow, column=maxcol).coordinate}"
table = Table(displayName="DataTable", ref=table_range) # Define the table with the detected range
table.tableStyleInfo=TableStyleInfo(name="TableStyleMedium9", showFirstColumn=False,
                       showLastColumn=False, showRowStripes=True, showColumnStripes=True)
ws.add_table(table) # Add the table to the worksheet

#set up cell width
column_widths = [10,48, 11, 30, 30, 18, 18, 30, 14, 18, 15]
columns = list('ABCDEFGHIJK')
for i, width in enumerate(column_widths):
    ws.column_dimensions[columns[i]].width = width

In [82]:
#save as excel
if deployment=='PBACO':
    wb.save(fr'{path}\patient_alerts_{date_input}\App Subscription_{date_input}.xlsx')
else:
    wb.save(fr'{path}\patient_alerts_{deployment}_{date_input}\App Subscription_{deployment}_{date_input}.xlsx')
wb.close()

#move to raw files
os.makedirs(fr'{path}\Raw_Files', exist_ok=True)
shutil.move(fr"{path}\App Subscription.xlsx", fr"{path}\Raw_Files\App Subscription.xlsx")

'C:\\Users\\ZemingLiu\\OneDrive - PBACO\\Documents\\Weekly App Reports\\Derrell Macros\\Patient Alerts Macro\\1223-1229 Empire\\Raw_Files\\App Subscription.xlsx'

In [83]:
continue_input=input('Enter Y when you are ready to continue')
print(fr"Part1 App Subcription is done, file saved at {path}\patient_alerts_{date_input}\App Subscription_{date_input}.xlsx")
if  continue_input.lower()=='y':
    print('Start Part2')
else:
    sys.exit()

Enter Y when you are ready to continue y


Part1 App Subcription is done, file saved at C:\Users\ZemingLiu\OneDrive - PBACO\Documents\Weekly App Reports\Derrell Macros\Patient Alerts Macro\1223-1229 Empire\patient_alerts_1223-1229\App Subscription_1223-1229.xlsx
Start Part2


# **Part 2 App Dashboard**

In [84]:
#copy file
import xlwings as xw
from datetime import datetime
if deployment == 'PBACO':
    source_file_path=fr"C:\Users\ZemingLiu\OneDrive - PBACO\Documents\Weekly App Reports\Derrell Macros\Patient Alerts Macro\{lastweek_date_input} {deployment}\app_usage_dashboard_{lastweek_date_input}.xlsx"
    destination_file_path=fr"C:\Users\ZemingLiu\OneDrive - PBACO\Documents\Weekly App Reports\Derrell Macros\Patient Alerts Macro\{date_input} {deployment}\app_usage_dashboard_{date_input}.xlsx"
else:
    source_file_path=fr"C:\Users\ZemingLiu\OneDrive - PBACO\Documents\Weekly App Reports\Derrell Macros\Patient Alerts Macro\{lastweek_date_input} {deployment}\app_usage_dashboard_{deployment}_{lastweek_date_input}.xlsx"
    destination_file_path=fr"C:\Users\ZemingLiu\OneDrive - PBACO\Documents\Weekly App Reports\Derrell Macros\Patient Alerts Macro\{date_input} {deployment}\app_usage_dashboard_{deployment}_{date_input}.xlsx"
shutil.copy(source_file_path,destination_file_path)

'C:\\Users\\ZemingLiu\\OneDrive - PBACO\\Documents\\Weekly App Reports\\Derrell Macros\\Patient Alerts Macro\\1223-1229 Empire\\app_usage_dashboard_Empire_1223-1229.xlsx'

In [85]:
#add a new sheet under current week
app = xw.App(visible=True)
wb = app.books.open(destination_file_path)
sheet_to_copy = wb.sheets[lastweek_date_input]
ws=sheet_to_copy.copy(after=sheet_to_copy)
ws.name = date_input

In [86]:
#rolling last 3 weeks value
for i in range(25, 1000):
    if ws.range(f"F{i}").value and 'Pts With Actions (%) Aggregate' in ws.range(f"F{i}").value:
        max_row = i
        break

source_range = ws.range(f'H25:J{max_row}')
destination_range = ws.range(f'G25:I{max_row}')
destination_range.value = source_range.value

In [87]:
#action ratio by pc
df_ratio_bypc = df.groupby('Provider Consultant').apply(lambda x: x['Total Pts With Actions'].sum() / x['Pts Sent To App'].sum()).reset_index(name='actions_per_consultant_ratio')
source_values = ws.range(f'F26:F{max_row}').value  # all pc
lookup_results = []
for value in source_values:
    if value in df_ratio_bypc['Provider Consultant'].values:
        action_ratio = df_ratio_bypc[df_ratio_bypc['Provider Consultant'] == value]['actions_per_consultant_ratio'].iloc[0]
        lookup_results.append(action_ratio)
    else:
        lookup_results.append(None)  # Append None if value is not found
lookup_results = [[result] for result in lookup_results] #change the list from horizional to vertical

# Write the results back to Excel in the range J26:J66
ws.range('J25').value = fr"{date_input[:2]}/{date_input[2:4]}-{date_input[5:7]}/{date_input[7:9]}"
ws.range(f'J26:J{max_row}').value = lookup_results 
ws.range(f'J{max_row}').value = f"{round((df[df['Provider Consultant']!='David T']['Total Pts With Actions'].sum()/df[df['Provider Consultant']!='David T']['Pts Sent To App'].sum()),4)}"

In [88]:
#change the header
ws.range('A1').value = fr"{deployment} APP Usage Dashboard - From: {date_input[:2]}/{date_input[2:4]}/{datetime.now().year} To: {date_input[5:7]}/{date_input[7:9]}/{datetime.now().year}"

#sort the data with desc ration
data_range = ws.range(f'F25:J{max_row-1}')
sort = ws.api.Sort
sort.SortFields.Clear()
sort.SortFields.Add(
    Key=ws.range(f'J25:J{max_row-1}').api,  # Sort by column J
    SortOn=0,  # xlSortOnValues (0 for values)
    Order=2,  # xlDescending (2 for descending order, 1 for ascending)
    DataOption=0  # xlSortNormal
)
sort.SetRange(ws.range(f'F25:J{max_row-1}').api)  # Define the entire range to sort
sort.Header = 0  # No header row in the range
sort.MatchCase = False
sort.Orientation = 1  # xlSortColumns
sort.SortMethod = 1  # xlPinYin
sort.Apply()

In [89]:
#save the file and print it to pdf
if deployment == 'PBACO':
    ws.api.ExportAsFixedFormat(0, fr"{path}\patient_alerts_{date_input}\app_usage_dashboard_{date_input}.pdf")
    wb.save()
else:
    ws.api.ExportAsFixedFormat(0, fr"{path}\patient_alerts_{deployment}_{date_input}\app_usage_dashboard_{deployment}_{date_input}.pdf")
    wb.save() 
wb.close()

In [90]:
continue_input=input('Enter Y when you are ready to continue')
print(fr"Part2 App Dashboard is done, file saved at {path}\patient_alerts_{date_input}\app_usage_dashboard_{date_input}.pdf")
if continue_input.lower()=='y':
    print('Start Part3')
else:
    sys.exit()

Enter Y when you are ready to continue y


Part2 App Dashboard is done, file saved at C:\Users\ZemingLiu\OneDrive - PBACO\Documents\Weekly App Reports\Derrell Macros\Patient Alerts Macro\1223-1229 Empire\patient_alerts_1223-1229\app_usage_dashboard_1223-1229.pdf
Start Part3


# **Part 3 Low App Performers**

In [91]:
#open workbook
if deployment == 'PBACO':
    source_file_path=fr"C:\Users\ZemingLiu\OneDrive - PBACO\Documents\Weekly App Reports\Derrell Macros\Patient Alerts Macro\{lastweek_date_input} {deployment}\Low_App_Performers_ongoing_{lastweek_date_input}.xlsx"
    destination_file_path=fr"C:\Users\ZemingLiu\OneDrive - PBACO\Documents\Weekly App Reports\Derrell Macros\Patient Alerts Macro\{date_input} {deployment}\Low_App_Performers_ongoing_{date_input}.xlsx"
else:
    source_file_path=fr"C:\Users\ZemingLiu\OneDrive - PBACO\Documents\Weekly App Reports\Derrell Macros\Patient Alerts Macro\{lastweek_date_input} {deployment}\Low_App_Performers_ongoing_{deployment}_{lastweek_date_input}.xlsx"
    destination_file_path=fr"C:\Users\ZemingLiu\OneDrive - PBACO\Documents\Weekly App Reports\Derrell Macros\Patient Alerts Macro\{date_input} {deployment}\Low_App_Performers_ongoing_{deployment}_{date_input}.xlsx"

In [92]:
#add new sheet under current week
shutil.copy(source_file_path,destination_file_path)
wb=load_workbook(destination_file_path, keep_links=False)
sheet_to_copy=wb[fr"{lastweek_date_input}"]
new_sheet=wb.copy_worksheet(sheet_to_copy)
new_sheet.title=date_input
new_sheet.delete_rows(2,amount=new_sheet.max_row)
new_sheet.auto_filter.ref = f"A1:{chr(64 + new_sheet.max_column)}1" #add filter

In [93]:
#reference value from last week
df_lowperformer=df[df['Pts With Actions(%)'].str.rstrip('%').astype('float')<=60][['TIN','TIN Name','NPI','Provider','Provider Consultant','In Pes','Pts Sent To App','Pts With Actions(%)','Total Pts With Actions','Logins']]
#df_lowperformer=df[df['Pts With Actions(%)']<=0.6][['TIN','TIN Name','NPI','Provider','Provider Consultant','In Pes','Pts Sent To App','Pts With Actions(%)','Total Pts With Actions','Logins']]
df_lastweek_lowperformer=pd.read_excel(source_file_path,sheet_name=lastweek_date_input)
merged_df=pd.merge(df_lowperformer,df_lastweek_lowperformer[['NPI','Consecutive Weeks']],on='NPI',how='left',suffixes=('', '_lastweek'))

#get value for this week
merged_df['Consecutive Weeks']=merged_df['Consecutive Weeks'].fillna(0)
merged_df['Consecutive Weeks']=merged_df['Consecutive Weeks']+1
merged_df['Last Week?']='NO'
merged_df.loc[merged_df['Consecutive Weeks']>1,'Last Week?']='YES' #avoid modifying on the slicer
merged_df['Notes']=''

#set up a TIN consecutive week='' Chantel's request
merged_df[merged_df['TIN'].astype('Int64')==824997767]['Consecutive Weeks']=''

In [94]:
#add value into the sheet
for r_idx, row in enumerate(merged_df.values, start=1):  # Start from row 2 to leave room for headers
    for c_idx, value in enumerate(row, start=1):  # Start from column 1 (A)
        new_sheet.cell(row=r_idx+1, column=c_idx, value=value)

In [95]:
#fill colors and set font
for cell in new_sheet[1]:  # Iterate over each cell in row 1
    cell.font = Font(name='Calibri', size=11, bold=True)

pt_sent_to_app_fill = PatternFill(start_color='6495ED', end_color='6495ED', fill_type='solid')
for row in range(1, new_sheet.max_row + 1):  # Use max_row to iterate through actual data rows
    new_sheet.cell(row=row, column=5).fill = pt_sent_to_app_fill
    
pt_with_actions_fill = PatternFill(start_color='90EE90', end_color='90EE90', fill_type='solid')
for col in range(6, 8):  
    for row in range(1, new_sheet.max_row + 1):
        new_sheet.cell(row=row, column=col).fill = pt_with_actions_fill

login_fill = PatternFill(start_color='AFEEEE', end_color='AFEEEE', fill_type='solid')
for row in range(1, new_sheet.max_row + 1):
    new_sheet.cell(row=row, column=8).fill = login_fill
    
thin_border = Border(left=Side(style='thin'),right=Side(style='thin'),top=Side(style='thin'),bottom=Side(style='thin'))
for row in new_sheet.iter_rows(min_row=1,max_row=new_sheet.max_row,min_col=1,max_col=new_sheet.max_column):
    for cell in row:
        cell.border = thin_border
        
#highlight 3 and 4 consecutive weeks
for row in range(2, new_sheet.max_row + 1): 
    cell = new_sheet[f"I{row}"]
    if new_sheet[f"I{row}"].value in [3,4]:
        cell.fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")
        cell.alignment = Alignment(horizontal='left')
        
for row in range(1,new_sheet.max_row+1):
    for cell in new_sheet[row]:
        

wb.save(destination_file_path)
wb.close()

In [96]:
#save the currentweek low performer spreedsheet as a new workbook, not support by openpyxl, will use xlwings to do it
wb = app.books.open(destination_file_path)
sheet_to_copy = wb.sheets[date_input]

new_book = xw.Book()
new_sheet = new_book.sheets[0]
sheet_to_copy.api.Copy(Before=new_sheet.api) 
copied_sheet = new_book.sheets[-1]
new_sheet.delete()

if deployment=='PBACO':
    new_book.save(fr"C:\Users\ZemingLiu\OneDrive - PBACO\Documents\Weekly App Reports\Derrell Macros\Patient Alerts Macro\{date_input} {deployment}\Low_App_Performers_{date_input}.xlsx")
else:
    new_book.save(fr"C:\Users\ZemingLiu\OneDrive - PBACO\Documents\Weekly App Reports\Derrell Macros\Patient Alerts Macro\{date_input} {deployment}\Low_App_Performers_{deployment}_{date_input}.xlsx")
    
wb.close()
new_book.close()
app.quit()

In [97]:
continue_input=input('Enter Y when you are ready to continue')
print(fr"Low App Performers is done, file saved at C:\Users\ZemingLiu\OneDrive - PBACO\Documents\Weekly App Reports\Derrell Macros\Patient Alerts Macro\{date_input} {deployment}\Low_App_Performers_{date_input}.xlsx")
if  continue_input.lower()=='y':
    print('Start Part4 Move Files')
else:
    sys.exit()

Enter Y when you are ready to continue y


Low App Performers is done, file saved at C:\Users\ZemingLiu\OneDrive - PBACO\Documents\Weekly App Reports\Derrell Macros\Patient Alerts Macro\1223-1229 Empire\Low_App_Performers_1223-1229.xlsx
Start Part4 Move Files


# **Part 4 Move Patients Alerts for each PC, App Subscription, App Usage Dashboard PDF to R Drive**

In [98]:
if deployment == 'PBACO':
    source_folder_path=fr'{path}\patient_alerts_{date_input}'
    destination_folder_path=fr'R:\Patient Alerts by Subscribed NPI\{date_input}'
else:
    source_folder_path=fr'{path}\patient_alerts_{deployment}_{date_input}'
    destination_folder_path=fr'R:\Patient Alerts by Subscribed NPI - {deployment}\{date_input}'
os.makedirs(destination_folder_path, exist_ok=True)

In [99]:
for filename in os.listdir(source_folder_path):
    if filename.lower().endswith('.pdf') or 'App Subscription' in filename or 'patient_alerts_All' in filename:
        source_file_path = os.path.join(source_folder_path,filename)
        destination_file_path=os.path.join(destination_folder_path,filename)
        shutil.copy(source_file_path,destination_file_path)
print('Move files done, all the files have been moved to {destination_folder_path}')

Move files done, all the files have been moved to {destination_folder_path}
